In [1]:
import pandas as pd
import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("data/image_labels.csv")
data

,image,label
0,data/apparels/Skirt/f4b934fc-e326-4f72-b535-53...,Skirt
1,data/apparels/Skirt/fdbcfdcd-5deb-4a20-9aef-b7...,Skirt
2,data/apparels/Skirt/ecfd00ef-2fe1-4778-a2a8-e9...,Skirt
3,data/apparels/Skirt/2ac96388-4353-44b6-a174-de...,Skirt
4,data/apparels/Skirt/bd289469-607c-45df-bad3-ed...,Skirt
...,...,...
11682,data/apparels/Shirt/7be93c0fe39fd61ea0c5217ea9...,Shirt
11683,data/apparels/Shirt/45279e1f-82af-4fe3-b0ad-2b...,Shirt
11684,data/apparels/Shirt/aaeccde537821c284988d61777...,Shirt
11685,data/apparels/Shirt/ae9cec7a-dd1d-49bc-adae-64...,Shirt


In [3]:
apparel_list = ['T-Shirt','Shirt', 'Dress', 'Pants', 'Shorts','Skirt']
apparel_dict = {'T-Shirt':0,'Shirt':1, 'Dress':2, 'Pants':3, 'Shorts':4,'Skirt':5}

In [4]:
data_label = data["label"].to_numpy()
boolean_data_label = [label == np.array(apparel_list) for label in data_label]

In [5]:
X = list(data["image"].values)
y = boolean_data_label

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7)

In [7]:
IMG_SIZE = 224

def process_image(image_path,img_size=IMG_SIZE):
    """
    Take an image file path and turn image into a Tensor.
    """
    image = tf.io.read_file(image_path) # Read image file
    image = tf.image.decode_jpeg(image,channels=3) # Turn the image into 3 channels RGB
    image = tf.image.convert_image_dtype(image,tf.float32) # Turn the value 0-255 to 0-1
    image = tf.image.resize(image,size=[img_size,img_size]) # Resize the image to 224x224
    return image 

In [8]:
def get_image_label(image_path,label):
    """
    Take an image file path name and the associated label,
    process the image and return a tuple of (image,label)
    """
    image = process_image(image_path)
    return image,label

In [9]:
BATCH_SIZE = 32

def create_data_batches(X,y=None,batch_size=BATCH_SIZE,valid_data=False,test_data=False):
    """
    Create batches of data out of image (X) and lebel (y) pairs.
    Shuffles the data if it's training data but doesn't shuffle if it's validation data.
    Also accepts test data as input (no labels).
    """
    # If the data is a test dataset, we probably don't have labels
    if test_data:
        print("Creating testing data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) # Only file path (no label)
        data_batch = data.map(process_image).batch(batch_size)
        return data_batch

    # If the data is valid dataset, we don't have to shuffle it
    elif valid_data:
        print("Creating validation data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y))) # file path, label
        data_batch = data.map(get_image_label).batch(batch_size)
        return data_batch

    else:
        print("Creating training data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y))) # file path, label
        data = data.shuffle(buffer_size=len(X))
        data_batch = data.map(get_image_label).batch(batch_size)
        return data_batch

In [10]:
# Create training and validation data batches
train_data = create_data_batches(X=X_train,y=y_train)
val_data = create_data_batches(X=X_test,y=y_test,valid_data=True)

Creating training data batches...
Creating validation data batches...


2022-04-20 13:16:41.063858: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
import matplotlib.pyplot as plt

def show_25_images(images,labels,predicted):
    """
    Display a plot of 25 images and their labels from a batch.
    """
    plt.figure(figsize=(15,10))
    for i in range(25):
        if apparel_list[labels[i].argmax()] == apparel_list[predicted[i].argmax()]:
            color = 'blue'
        else:
            color = 'red'
        ax = plt.subplot(5,5,i+1)
        plt.imshow(images[i])
        plt.title(apparel_list[labels[i].argmax()],color=color)
        plt.axis("off")

In [12]:
train_images,train_labels = next(train_data.as_numpy_iterator())
# show_25_images(train_images,train_labels,train_labels)

In [13]:
val_images,val_labels = next(val_data.as_numpy_iterator())
# show_25_images(val_images,val_labels,val_labels)

In [14]:
NUN_EPOCHS = 100
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=3)
INPUT_SHAPE = [None,IMG_SIZE,IMG_SIZE,3]
OUTPUT_SHAPE = 6
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

def create_model(input_shape=INPUT_SHAPE,output_shape=OUTPUT_SHAPE,model_url=MODEL_URL):
    print(f"Building model with: {model_url}")
    model = tf.keras.Sequential([
        hub.KerasLayer(model_url), # Layer 1: Input layer
        tf.keras.layers.Dense(units=OUTPUT_SHAPE,activation="softmax")]) # Layer 2: Output layer
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"])
    model.build(input_shape)
    return model

In [15]:
import datetime

def create_tensorboard_callback():
    logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir)

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=3)
NUM_EPOCHS = 20

def train_model():
    """
    Train a given model and return the trained version.
    """
    model = create_model() # Create a model
    model.summary()
    tensorboard = create_tensorboard_callback() # Create tensorboard callback
    model.fit(x=train_data,
              epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq=1,
            callbacks=[tensorboard,early_stopping]) # Fit the model passing it the callback we created
    return model

In [ ]:
model = train_model()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              5432713   
                                                                 
 dense (Dense)               (None, 6)                 6012      
                                                                 
Total params: 5,438,725
Trainable params: 6,012
Non-trainable params: 5,432,713
_________________________________________________________________
Epoch 1/20
293/293 [==============================] - 238s 802ms/step - loss: 0.4163 - accuracy: 0.8569 - val_loss: 0.2703 - val_accuracy: 0.9076
Epoch 2/20
293/293 [==============================] - 236s 806ms/step - loss: 0.2337 - accuracy: 0.9182 - val_loss: 0.2416 - val_accuracy: 0.9192
Epoch 3/20
293/293 [==============================] -

In [ ]:
# model.save("models/apparel-classification.h5")

In [ ]:
test_predicted = model.predict(val_data)

In [ ]:
list_predicted = [l.argmax() for l in np.round(test_predicted)]
list_y_test = [l.argmax() for l in y_test]

In [ ]:
from sklearn.metrics import accuracy_score

print(f"The accuracy: {accuracy_score(list_predicted,list_y_test)*100:.2f}")

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(list_y_test,list_predicted)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax,fmt='.0f',cmap='Oranges'); #annot=True to annotate cells

ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(list(apparel_list)); ax.yaxis.set_ticklabels(list(apparel_list));

In [ ]:
show_25_images(val_images,y_test,test_predicted)